In [1]:
import os,re,csv,sys,ast,json,time,urllib,random,spotipy,requests,datetime
import numpy as np
import pandas as pd
import urllib.parse
from bs4 import BeautifulSoup
from user_agent import generate_user_agent
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# spotifyAPI存取權限
client_id = '1a23ed1ca359485ab81d524206dba0f6'
client_secret = 'af0e14fdb19a49dfb19ef0c1ea1d42ea'
username = 'qnwv65t11cplaz4dikhl4mjgi'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
# 檔案名稱所有非法字元都轉義：https://www.itread01.com/content/1549125182.html
def validateTitle(title):
    rstr = r"[\/\\\:\*\?\"\<\>\|]" # '/ \ : * ? " < > |'
    new_title = re.sub(rstr, " ", title)  # 替換為空白
    return new_title

In [4]:
# 取得歌手的所有專輯
def show_artist_albums(artist_id):
    albums = []
    # album_type有四種：album、single、appears_on、compilation (include_groups=album,single)
    results = sp.artist_albums(artist_id, album_type='album')
    albums.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    # 移除重複的album
    seen = set()
    # 按照專輯的名稱排序
    albums.sort(key=lambda album: album['name'].lower())
    list_album_name = []
    list_album_id = []
    list_album_uri = []
    list_album_href = []
    for album in albums:
        album_name = album['name']
        album_id = album['id']
        album_uri = album['uri']
        album_href = 'https://open.spotify.com/album/' + album_id
        if album_name not in seen:
            #print(album_name)
            seen.add(album_name)
            list_album_name.append(album_name)
            list_album_id.append(album_id)
            list_album_uri.append(album_uri)
            list_album_href.append(album_href)
    df_albums = pd.DataFrame(list(zip(list_album_name,list_album_id,list_album_uri,list_album_href)), columns =['Name','Id','Uri','Href'])
    df_albums.drop_duplicates(subset='Name', keep='first', inplace=True)
    df_albums.reset_index(drop=True, inplace=True)
    return df_albums

In [5]:
# 每位歌手抓取專輯資料並輸出csv(df_albums_+ artist_name +.csv)
empty_album = [] #沒有專輯的歌手
exist_album = [] #有專輯的歌手

df_id_genres = pd.read_csv(r'csv/20200303/v3/artist_list_ID+Genres_select_v3.csv')
for i in range(len(df_id_genres)):
    artist_name = df_id_genres['Artist'][i]
    artist_id = df_id_genres['Id'][i]
    
    df_albums = show_artist_albums(artist_id)
    # 歌手沒有專輯data
    if df_albums.empty == True:
        empty_album.append(artist_name) #沒有專輯data的歌手名稱放入empty_album列表中
        print(str(i) + ' | ' + str(artist_name) + ' | ' + 'N')
    # 歌手有專輯data
    if df_albums.empty == False:
        exist_album.append(artist_name) #有專輯data的歌手名稱放入exist_album列表中
        artist_name = validateTitle(artist_name) #檔名去除無效字元
        df_albums.to_csv('csv/20200303/v3/v3_albums/df_albums_'+ artist_name +'.csv',index=0,encoding="utf_8_sig")
        print(str(i) + ' | ' + str(artist_name) + ' | ' + 'Y')

print('DONE')

0 | Adam Joseph | Y
1 | Aisha Burns | Y
2 | Akon | Y
3 | Alec Ounsworth | Y
4 | Alex Goot | Y
5 | Alfred Apaka | Y
6 | Ali Brustofski | Y
7 | Alice Gerrard | Y
8 | Alice Peacock | Y
9 | Alison Gold | N
10 | Alissa Moreno | Y
11 | Alyson  | N
12 | Andrew Osenga | Y
13 | Andrew Rowan Summers | Y
14 | Andy Williams | Y
15 | Anita Bryant | Y
16 | Anna Oxygen | Y
17 | Annet Artani | Y
18 | Annette Funicello | Y
19 | Annie LeBlanc | N
20 | Anthony Kellman | Y
21 | Ashlee Simpson | Y
22 | Ashley Roberts | Y
23 | Austin Mahone | Y
24 | Bazzi  | Y
25 | BC Jean | N
26 | Beckah Shae | Y
27 | Bella Thorne | N
28 | Ben Harper | Y
29 | Betty Johnson | Y
30 | Billy Joe Royal | Y
31 | Billy Swan | Y
32 | Bo Bice | Y
33 | Bob Childers | Y
34 | Bob Manning | Y
35 | Bobby Helms | Y
36 | Bradley Hathaway | Y
37 | Brandi Emma | N
38 | Bree Sharp | Y
39 | Brie Larson | Y
40 | Britton Buchanan | Y
41 | Brooke Hogan | Y
42 | Brooke White | Y
43 | Caitlin Cary | Y
44 | Caitlin Rose | Y
45 | Cal Smith | Y
46 | 